In [1]:
import numpy as np # linear algebra
import scipy as scipy
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import dill

import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

import cv2
from tqdm import tqdm

from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPooling2D, UpSampling2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.layers.merge import concatenate
import math
from keras.callbacks import ModelCheckpoint



Using Theano backend.


In [2]:
label_map = {'agriculture': 4, 'selective_logging': 0, 'primary': 1, 'blow_down': 10, 'cloudy': 3, 'slash_burn': 2, 'artisinal_mine': 5, 'clear': 6, 'cultivation': 7, 'conventional_mine': 8, 'habitation': 9, 'road': 11, 'haze': 12, 'blooming': 14, 'water': 15, 'partly_cloudy': 16, 'bare_ground': 13}

In [4]:
x_test = []

for i in range(40669):
    img = cv2.imread('../../DB/test-jpg/test_{}.jpg'.format(i))
    x_test.append(cv2.resize(img, (72, 72)))
    
    
for i in range(20522):
    img = cv2.imread('../../DB/test-jpg-additional/file_{}.jpg'.format(i))
    x_test.append(cv2.resize(img, (72, 72)))

In [5]:
x_test = np.array(x_test, np.float16) / 255.

In [6]:
input_layer = Input(shape=(72, 72, 3))
batch1 = BatchNormalization()(input_layer)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(batch1)
conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
max1 = MaxPooling2D(pool_size=(2, 2))(conv2)
drop1 = Dropout(0.3)(max1)

batch2 = BatchNormalization()(drop1)
conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(batch2)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv3)
max2 = MaxPooling2D(pool_size=(2, 2))(conv4)
drop2 = Dropout(0.3)(max2)

batch3 = BatchNormalization()(drop2)
conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(batch3)
conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)
max3 = MaxPooling2D(pool_size=(2, 2))(conv6)
drop3 = Dropout(0.3)(max3)

batch4 = BatchNormalization()(drop3)
conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(batch4)
conv8 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)

globAv1 = GlobalAveragePooling2D()(drop2)
globAv2 = GlobalAveragePooling2D()(drop3)
globAv3 = GlobalAveragePooling2D()(conv8)

conc = concatenate([globAv1, globAv2, globAv3])

dense1 = Dense(512, activation='relu')(conc)
drop4 = Dropout(0.5)(dense1)
dense2 = Dense(17, activation='sigmoid')(drop4)
unet = Model(inputs=[input_layer], outputs=dense2)

unet.summary()

unet.load_weights('run8_weights.hdf5')

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 72, 72, 3)     0                                            
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 72, 72, 3)     12          input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 72, 72, 32)    896         batch_normalization_1[0][0]      
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 72, 72, 32)    9248        conv2d_1[0][0]                   
___________________________________________________________________________________________

In [8]:
predictions = unet.predict(x_test)

In [9]:
def map_predictions(predictions, labels_map, thresholds):
    """
    Return the predictions mapped to their labels
    :param predictions: the predictions from the predict() method
    :param labels_map: the map
    :param thresholds: The threshold of each class to be considered as existing or not existing
    :return: the predictions list mapped to their labels
    """
    predictions_labels = []
    for prediction in predictions:
        labels = [labels_map[i] for i, value in enumerate(prediction) if value > thresholds[i]]
        predictions_labels.append(labels)

    return predictions_labels

In [10]:
inv_label_map = {value: key for key, value in label_map.items()}
thres = [0.2] * len(label_map)
predicted_labels = map_predictions(predictions, inv_label_map, thres)

x_test_filename = ['test_{}.jpg'.format(i) for i in range(40669)]
x_test_filename2 = ['file_{}.jpg'.format(i) for i in range(20522)]
x_test_filename = np.hstack((x_test_filename, x_test_filename2))
print(x_test_filename.shape)

(61191,)


In [11]:
tags_list = [None] * len(predicted_labels)
for i, tags in enumerate(predicted_labels):
    tags_list[i] = ' '.join(map(str, tags))

final_data = [[filename.split(".")[0], tags] for filename, tags in zip(x_test_filename, tags_list)]

final_df = pd.DataFrame(final_data, columns=['image_name', 'tags'])
final_df.head()

final_df.to_csv('run8.csv', index=False)

got 0.92090^

below: optimizing thresholds

In [12]:
from sklearn.metrics import fbeta_score

def optimise_f2_thresholds(y, p, verbose=True, resolution=100):
  def mf(x):
    p2 = np.zeros_like(p)
    for i in range(17):
      p2[:, i] = (p[:, i] > x[i]).astype(np.int)
    score = fbeta_score(y, p2, beta=2, average='samples')
    return score

  x = [0.2]*17
  for i in range(17):
    best_i2 = 0
    best_score = 0
    for i2 in range(resolution):
      i2 /= resolution
      x[i] = i2
      score = mf(x)
      if score > best_score:
        best_i2 = i2
        best_score = score
    x[i] = best_i2
    if verbose:
      print(i, best_i2, best_score)

  return x

In [15]:
x_train = []
y_train = []

df_train = pd.read_csv('../../DB/train_v2.csv')

for f, tags in tqdm(df_train.values, miniters=1000):
    img = cv2.imread('../../DB/train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train.append(cv2.resize(img, (72, 72)))
    y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float16) / 255.

print(x_train.shape)
print(y_train.shape)

split = 35000
x_train, x_valid, y_train, y_valid = x_train[:split], x_train[split:], y_train[:split], y_train[split:]

100%|██████████| 40479/40479 [11:48<00:00, 57.17it/s]


(40479, 72, 72, 3)
(40479, 17)


In [16]:
x_valid_pred = unet.predict(x_valid)
opt_thresh = optimise_f2_thresholds(y_valid, x_valid_pred)

0 0.12 0.921845831959
1 0.23 0.921983458979
2 0.18 0.92202039655


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


3 0.09 0.922995252029
4 0.17 0.923018840605
5 0.1 0.923021273002
6 0.19 0.923099722974
7 0.26 0.923279553869
8 0.3 0.92330304242
9 0.18 0.923423868258
10 0.21 0.923432815075
11 0.21 0.923498359135
12 0.23 0.923591710599
13 0.19 0.923702622196
14 0.23 0.923756349572
15 0.2 0.923756349572
16 0.26 0.923865217908


In [18]:
predicted_labels_new_thresh = []

for prediction in predictions:
    labels = [inv_label_map[i] for i, value in enumerate(prediction) if value > opt_thresh[i]]
    predicted_labels_new_thresh.append(labels)

    
tags_list = [None] * len(predicted_labels_new_thresh)
for i, tags in enumerate(predicted_labels_new_thresh):
    tags_list[i] = ' '.join(map(str, tags))

final_data = [[filename.split(".")[0], tags] for filename, tags in zip(x_test_filename, tags_list)]

final_df = pd.DataFrame(final_data, columns=['image_name', 'tags'])
final_df.head()

final_df.to_csv('run8_opt_thresh.csv', index=False)

^ got 0.92053 for some reason

trying to run f-score with 0.2 thresholds

In [20]:
print(fbeta_score(y_valid, np.array(x_valid_pred) > 0.2, beta=2, average='samples'))

0.921802446485


trying optimized thresholds with lower resolution...

In [22]:
opt_thresh2 = optimise_f2_thresholds(y_valid, x_valid_pred, resolution=10)
predicted_labels_new_thresh = []

for prediction in predictions:
    labels = [inv_label_map[i] for i, value in enumerate(prediction) if value > opt_thresh2[i]]
    predicted_labels_new_thresh.append(labels)

    
tags_list = [None] * len(predicted_labels_new_thresh)
for i, tags in enumerate(predicted_labels_new_thresh):
    tags_list[i] = ' '.join(map(str, tags))

final_data = [[filename.split(".")[0], tags] for filename, tags in zip(x_test_filename, tags_list)]

final_df = pd.DataFrame(final_data, columns=['image_name', 'tags'])
final_df.head()

final_df.to_csv('run8_opt_thresh2.csv', index=False)

0 0.2 0.921802446485
1 0.2 0.921802446485
2 0.3 0.92182989404


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


3 0.1 0.922772552599
4 0.2 0.922772552599
5 0.1 0.922774184491
6 0.2 0.922774184491
7 0.2 0.922774184491
8 0.3 0.922795838438
9 0.3 0.922858352835
10 0.2 0.922858352835
11 0.2 0.922858352835
12 0.2 0.922858352835
13 0.1 0.922884819517
14 0.3 0.922916777326
15 0.2 0.922916777326
16 0.2 0.922916777326


got 0.92050^